In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from geopy.distance import geodesic 
import get_flight_data as gdf

### 1. list of all countries

In [5]:
countries_sub_url = (["avia_par_be.tsv.gz",
                      "avia_par_bg.tsv.gz",
                      "avia_par_cz.tsv.gz",
                      "avia_par_dk.tsv.gz",
                      "avia_par_de.tsv.gz",
                      "avia_par_ee.tsv.gz",
                      "avia_par_ie.tsv.gz",
                      "avia_par_el.tsv.gz",
                      "avia_par_es.tsv.gz",
                      "avia_par_fr.tsv.gz",
                      "avia_par_hr.tsv.gz",
                      "avia_par_it.tsv.gz",
                      "avia_par_cy.tsv.gz",
                      "avia_par_lv.tsv.gz",
                      "avia_par_lt.tsv.gz",
                      "avia_par_lu.tsv.gz",
                      "avia_par_hu.tsv.gz",
                      "avia_par_mt.tsv.gz",
                      "avia_par_nl.tsv.gz",
                      "avia_par_at.tsv.gz",
                      "avia_par_pl.tsv.gz",
                      "avia_par_pt.tsv.gz",
                      "avia_par_ro.tsv.gz",
                      "avia_par_si.tsv.gz",
                      "avia_par_sk.tsv.gz",
                      "avia_par_fi.tsv.gz",
                      "avia_par_se.tsv.gz",
                      "avia_par_is.tsv.gz",
                      "avia_par_no.tsv.gz",
                      "avia_par_ch.tsv.gz",
                      "avia_par_me.tsv.gz",
                      "avia_par_mk.tsv.gz",
                      "avia_par_tr.tsv.gz",
                      "avia_par_uk.tsv.gz"])

In [6]:
country_codes =   ['be',
                   'bg',
                   'dk',
                   'de',
                   'ee',
                   'ie',
                   'el',
                   'es',
                   'fr',
                   'hr',
                   'it',
                   'cy',
                   'lv',
                   'lt',
                   'lu',
                   'hu',
                   'mt',
                   'nl',
                   'at',
                   'pl',
                   'pt',
                   'ro',
                   'si',
                   'sk',
                   'fi',
                   'se',
                   'is',
                   'no',
                   'ch',
                   'me',
                   'mk',
                   'tr',
                   'uk']

country_codes = [code.upper() for code in country_codes]

### 2. define functions

#### data aggregation function

In [31]:
def make_agg_one_country(country_df,country_id):  
    
    df_by_year = (country_df[["flight_d","seat_d","passenger_d","distance","year"]]
                  .groupby(by="year")
                  .agg({"flight_d" : "sum",
                        "seat_d" : "sum",
                        "passenger_d": "sum",
                        "distance": "mean"}))
    
    
    
    # for the distances we do not want to account for domestic flights two times
    df_a = country_df.copy()#[["fr_country","to_country","year","month","distance"]].copy()
    df_a = df_a.sort_values(by = ["distance","month"])
    df_a["shift_dist"] = df_a["distance"].shift(1)
    df_a["shift_month"] = df_a["month"].shift(1)
    # now, if a connection appears twice, then dist=shif_dist and month=shift_month
    df_a["test"] =  ~((df_a["distance"] == df_a["shift_dist"]) & (df_a["month"] == df_a["shift_month"]))
    df_a = df_a.loc[df_a["test"],]
    
    
    # check wether flight is intra European
    df_a["european"] = df_a["to_country"].isin(country_codes).astype(int)

    
    # calculate the mean distance/euro-rate for each year
    df_a_by_year = (df_a[["year","distance","european"]]
                    .groupby(by="year")
                    .agg({"distance":np.mean,"european":np.mean}))     
    
   # put all info togherter in a output dictionary
    country_row = {("country_code",""): country_id,
                   ("airports",""):len(country_df["fr_airport"].value_counts())}
    
    for year in range(2003,2019):
        try:
            country_row[(str(year),"f")] = df_by_year.loc[year,"flight_d"]
            country_row[(str(year),"s")] = df_by_year.loc[year,"seat_d"]
            country_row[(str(year),"p")] = df_by_year.loc[year,"passenger_d"]
            country_row[(str(year),"avr_d")] = df_a_by_year.loc[year,"distance"]
            country_row[(str(year),"intra_euro")] = df_a_by_year.loc[year,"european"]
        except:
            country_row[(str(year),"f")] = np.nan
            country_row[(str(year),"s")] = np.nan
            country_row[(str(year),"p")] = np.nan
            country_row[(str(year),"avr_d")] = np.nan
            country_row[(str(year),"intra_euro")] = np.nan
        
    return country_row

#### calculating distances

In [9]:
def get_distance(icao1, icao2):

    try:
        coord1 = (df_geo.loc[df_geo['icao'] == icao1, 'latitude'].iloc[0],
                  df_geo.loc[df_geo['icao'] == icao1, 'longitude'].iloc[0])
        try:
            coord2 = (df_geo.loc[df_geo['icao'] == icao2, 'latitude'].iloc[0],
                      df_geo.loc[df_geo['icao'] == icao2, 'longitude'].iloc[0])
            dist = geodesic(coord1, coord2).km
        except:
            dist = np.nan
            missing_ports.append(icao2) ## global    
    except:
        dist = np.nan
        missing_ports.append(icao1)

    return dist

In [10]:
def add_dist_to_country_df(df, df_geo):
    # look for unique connections
    df_unique_conn = df.drop_duplicates(subset='route').copy()
    # add distance column
    df_unique_conn['distance'] = df_unique_conn.apply(lambda x: get_distance(x['fr_airport'], x['to_airport']), axis=1)
    # merge distance on all connections
    return df.merge(df_unique_conn[['route', 'distance']], how='left', on='route')

### 3. load data for all countries

In [11]:
path_to_data = "../data/"
df_geo = pd.read_csv(path_to_data + "world_airports.csv")
missing_ports = []

##### from url

In [176]:
multicolumns = [("country_code",""),("airports","")]
for year in range(2003,2019):
    multicolumns.append((str(year),"f"))
    multicolumns.append((str(year),"s"))
    multicolumns.append((str(year),"p"))
    multicolumns.append((str(year),"avr_d"))
    multicolumns.append((str(year),"intra_euro"))
    

                        
micolumns = pd.MultiIndex.from_tuples(multicolumns)
df_countries = pd.DataFrame(columns = micolumns)


# FOR TESTS: ["avia_par_ee.tsv.gz"]
for country in ountries_sub_url:
    # get and transform the data
    df_country = gdf.load_and_transform_data(country)
    
    country_name = df_country.loc[0,"fr_country"]
    
    # limit years
    df_country["year"] = df_country["month"].dt.year
    df_country = df_country.loc[df_country["year"].le(2018) & df_country["year"].ge(2003)] # take only years 2003-2018
    
    
    # add distances
    df_country = add_dist_to_country_df(df_country, df_geo)
    
    
    ##### backup save so we don not have to do that every time we change the agg funtion
    df_country.to_csv(f"../torsten_tests/data/{country_name}.csv",index=False)
    
    

    
    # in some countries we have flights from one airport to the same airport. this is clearly a mistake
    # in the CZ data the arrival airport names are messed up, so we have no distance info there
    if country_name != "CZ":
        df_country = df_country.loc[df_country["distance"] != 0,]
    
    
    # get aggregate dafr_countryta and append to all countries df
    
    df_countries = df_countries.append(make_agg_one_country(df_country,country_name),ignore_index=True)
    print(country_name)

CZ


##### from files

In [37]:
# so you do not have to load and rearrange the tables, you can just save and load them
saved_path = "../torsten_tests/data/"

file_names = [saved_path + f"{file_name.upper()}.csv" for file_name in country_codes]

In [38]:
multicolumns = [("country_code",""),("airports","")]
for year in range(2003,2019):
    multicolumns.append((str(year),"f"))
    multicolumns.append((str(year),"s"))
    multicolumns.append((str(year),"p"))
    multicolumns.append((str(year),"avr_d"))
    multicolumns.append((str(year),"intra_euro"))

                        
micolumns = pd.MultiIndex.from_tuples(multicolumns)
df_countries = pd.DataFrame(columns = micolumns)

for file in file_names:
    
    df_country = pd.read_csv(file)
    country_name = df_country.loc[0,"fr_country"]
    
    # in the CZ data the arrival airport names are messed up, so we have no distance info there
    if country_name != "CZ":
        df_country = df_country.loc[df_country["distance"] != 0,]
    
    # save results if needed
    #df_country.to_csv(saved_path + f"{file_name.upper()}.csv",index=False)
    
    df_countries = df_countries.append(make_agg_one_country(df_country,country_name),ignore_index=True)

In [35]:
df_countries

country_code airports      2003                                        \
                                 f            s           p        avr_d   
0            BE        5   75971.0    9358444.0   5950900.0  1872.636191   
1            BG        3       NaN          NaN         NaN          NaN   
2            DK        7  116353.0   15438784.0   8809356.0  1707.545503   
3            DE       31  671606.0   87024337.0  60100327.0  2114.920104   
4            EE        1       NaN          NaN         NaN          NaN   
5            IE        6   84133.0   13070997.0   8974586.0  1592.248397   
6            EL       23  134102.0    2617844.0  12744101.0  1794.542049   
7            ES       34  618704.0   90196368.0  65355616.0  1900.133335   
8            FR       53  602273.0   76744945.0  51625146.0  1986.317436   
9            HR        6       NaN          NaN         NaN          NaN   
10           IT       37  481964.0   65732343.0  42379375.0  1535.293713   
11           CY        2       NaN          NaN         NaN          NaN   
12           LV        1       NaN          NaN         NaN          NaN   
13           LT        3       NaN          NaN         NaN          NaN   
14           LU        1       NaN          NaN         NaN          NaN   
15           HU        2   22895.0          0.0   1639140.0  1048.833958   
16           MT        1   11323.0    1362752.0    973682.0  1503.364306   
17           NL        5  170918.0          0.0  18222361.0  2956.166753   
18           AT        6   89901.0    8741419.0   5712612.0  1608.189882   
19           PL       12       NaN          NaN         NaN          NaN   
20           PT       10       NaN          NaN         NaN          NaN   
21           RO        8       NaN          NaN         NaN          NaN   
22           SI        1       NaN          NaN         NaN          NaN   
23           SK        2       NaN          NaN         NaN          NaN   
24           FI       16   90709.0   10296441.0   5482321.0  1405.587632   
25           SE       22  160529.0   17461636.0  11169098.0  1467.982607   
26           IS        3    5038.0    1108989.0    677442.0  2893.237501   
27           NO       30  220140.0   22338597.0  13614001.0   699.006925   
28           CH        5       NaN          NaN         NaN          NaN   
29           ME        2       NaN          NaN         NaN          NaN   
30           MK        1       NaN          NaN         NaN          NaN   
31           TR       43       NaN          NaN         NaN          NaN   
32           UK       37  797972.0  119286667.0  88851523.0  2095.010962   

                   2004                           ...      2017               \
   intra_euro         f            s           p  ...         f            s   
0    0.783459   81767.0   10282024.0   7037871.0  ...  106713.0   18020673.0   
1         NaN       NaN          NaN         NaN  ...   29743.0    5044006.0   
2    0.879397  133369.0   16925612.0   9830596.0  ...  135942.0   20068087.0   
3    0.794899  731203.0   93928794.0  66692915.0  ...  819202.0  134125308.0   
4         NaN       NaN          NaN         NaN  ...   14200.0    1449416.0   
5    0.852211   86274.0   13483925.0   9564340.0  ...  109166.0   18499806.0   
6    0.926377  148936.0     275248.0  13767047.0  ...       NaN          NaN   
7    0.942475  677622.0  109234751.0  72048428.0  ...  766642.0  130522087.0   
8    0.747634  594938.0   78943924.0  54150037.0  ...  618711.0  101990032.0   
9         NaN       NaN          NaN         NaN  ...   32678.0    4519091.0   
10   0.832774  492505.0   69758354.0  45933986.0  ...  562429.0   90022942.0   
11        NaN    9266.0    1595167.0   1225710.0  ...   26478.0    4864790.0   
12        NaN    9218.0     727838.0    431560.0  ...   23806.0    2599591.0   
13        NaN       NaN          NaN         NaN  ...   18519.0    2613735.0   
14        NaN       NaN          NaN         NaN  ...   19631.

### 4. save data

In [36]:
df_countries.to_csv("../data/aggregate_data.csv",index = False)

### 5. special case greece (EL) --- OLD

In [8]:
# load data and have a look
df_country = gdf.filter_out_arr_dep(gdf.make_df_from_sub_url("avia_par_el.tsv.gz"))

In [13]:
# compare seats and passengers
df_country["type1"].value_counts()

PAS       86004
FLIGHT    39652
SEAT      11977
Name: type1, dtype: int64

In [16]:
gdf.merge_to_final_df(df_country)

,flight,fr_country,fr_airport,to_country,to_airport,date,num_flights,seats,passengers
0,EL_LGAL_EL_LGAV,EL,LGAL,EL,LGAV,2019-03-01,230.0,1808.0,17964.0
1,EL_LGAV_AE_OMDB,EL,LGAV,AE,OMDB,2019-03-01,63.0,1225.0,16466.0
2,EL_LGAV_AT_LOWW,EL,LGAV,AT,LOWW,2019-03-01,157.0,2084.0,21470.0
3,EL_LGAV_BE_EBBR,EL,LGAV,BE,EBBR,2019-03-01,116.0,1034.0,17568.0
4,EL_LGAV_BG_LBSF,EL,LGAV,BG,LBSF,2019-03-01,136.0,2092.0,19422.0
...,...,...,...,...,...,...,...,...,...
11972,EL_LGTS_NL_EHAM,EL,LGTS,NL,EHAM,2003-01-01,64.0,1000.0,5231.0
11973,EL_LGTS_RU_UUDD,EL,LGTS,RU,UUDD,2003-01-01,16.0,1154.0,2476.0
11974,EL_LGTS_UK_EGCC,EL,LGTS,UK,EGCC,2003-01-01,2.0,500.0,59.0
11975,EL_LGTS_UK_EGLC,EL,LGTS,UK,EGLC,2003-01-01,44.0,1210.0,2859.0


apparently, the info on the number of seats is not available for a large number of flights. Moreover, even the existing seat information seem to be wrong (are often lower than the number of passengers).  Hence, can not trust the greece data and drop them

In [20]:
df_countries = df_countries.loc[~df_countries["country"].eq("EL"),]

In [22]:
df_countries.to_csv("aggregate_data.csv",index = False)